#  PHASE 3: Feature Engineering

## Big-Tech-Grade User Retention & Churn Prediction System

---

**Author**: Senior Data Scientist  
**Date**: February 2026  
**Objective**: Engineer behavioral features with business rationale for churn prediction

---

##  Feature Categories
1. **Recency Features** - Time since last activity
2. **Frequency Features** - Purchase patterns
3. **Monetary Features** - Spending behavior
4. **Trend Features** - Behavioral changes over time
5. **Engagement Features** - Product and basket patterns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print("✅ Libraries imported successfully")

In [ ]:
import os

PROJECT_ROOT = '/Users/anuj/Desktop/Churn_Retension/churn-prediction-bigtech'
PROCESSED_DATA_PATH = os.path.join(PROJECT_ROOT, 'data', 'processed')

df = pd.read_parquet(os.path.join(PROCESSED_DATA_PATH, 'transactions_clean.parquet'))
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

OBSERVATION_DATE = pd.Timestamp('2011-10-10')
CHURN_WINDOW = 60

print(f"✅ Loaded {len(df):,} transactions")
print(f"   Date range: {df['InvoiceDate'].min().date()} to {df['InvoiceDate'].max().date()}")
print(f"   Observation date: {OBSERVATION_DATE.date()}")

In [ ]:
df_hist = df[df['InvoiceDate'] <= OBSERVATION_DATE].copy()

df_hist['Revenue'] = df_hist['Quantity'] * df_hist['Price']

print(f"Historical data for feature engineering: {len(df_hist):,} transactions")
print(f"Unique customers: {df_hist['Customer ID'].nunique():,}")

---

## 1. Recency Features

**Business Rationale**: Recency is the strongest predictor of churn. Customers who haven't purchased recently are more likely to churn.

In [ ]:
print("="*60)
print("1. RECENCY FEATURES")
print("="*60)

recency_features = df_hist.groupby('Customer ID').agg(
    last_purchase_date=('InvoiceDate', 'max'),
    first_purchase_date=('InvoiceDate', 'min')
).reset_index()

recency_features['recency_days'] = (OBSERVATION_DATE - recency_features['last_purchase_date']).dt.days

recency_features['tenure_days'] = (OBSERVATION_DATE - recency_features['first_purchase_date']).dt.days

recency_features['active_period_days'] = (recency_features['last_purchase_date'] - recency_features['first_purchase_date']).dt.days

recency_features['recency_ratio'] = recency_features['recency_days'] / recency_features['tenure_days'].clip(lower=1)

recency_features['is_recent_30d'] = (recency_features['recency_days'] <= 30).astype(int)

print("\nRecency Features Created:")
print("-" * 50)
feature_explanations = {
    'recency_days': 'Days since last purchase - Primary churn indicator',
    'tenure_days': 'Customer tenure - Longer tenure = more invested',
    'active_period_days': 'Time between first and last purchase',
    'recency_ratio': 'Recency relative to tenure - Normalized measure',
    'is_recent_30d': 'Binary: purchased in last 30 days'
}
for feat, expl in feature_explanations.items():
    print(f"   {feat}: {expl}")

---

## 2. Frequency Features

**Business Rationale**: Purchase frequency indicates engagement level. Frequent buyers are more committed to the brand.

In [ ]:
print("="*60)
print("2. FREQUENCY FEATURES")
print("="*60)

frequency_features = df_hist.groupby('Customer ID').agg(
    total_orders=('Invoice', 'nunique'),
    
    total_transactions=('Invoice', 'count'),
    
    active_days=('InvoiceDate', lambda x: x.dt.date.nunique()),
    
    first_purchase=('InvoiceDate', 'min'),
    last_purchase=('InvoiceDate', 'max')
).reset_index()

frequency_features['tenure_months'] = ((OBSERVATION_DATE - frequency_features['first_purchase']).dt.days / 30).clip(lower=0.5)
frequency_features['orders_per_month'] = frequency_features['total_orders'] / frequency_features['tenure_months']

frequency_features['active_period'] = (frequency_features['last_purchase'] - frequency_features['first_purchase']).dt.days
frequency_features['avg_days_between_orders'] = frequency_features['active_period'] / (frequency_features['total_orders'] - 1).clip(lower=1)

def calculate_purchase_regularity(group):
    """Calculate standard deviation of days between purchases"""
    dates = group['InvoiceDate'].dt.normalize().drop_duplicates().sort_values()
    if len(dates) < 2:
        return np.nan
    intervals = dates.diff().dt.days.dropna()
    return intervals.std() if len(intervals) > 0 else np.nan

purchase_regularity = df_hist.groupby('Customer ID').apply(calculate_purchase_regularity).reset_index()
purchase_regularity.columns = ['Customer ID', 'purchase_interval_std']

frequency_features = frequency_features.merge(purchase_regularity, on='Customer ID', how='left')

frequency_features['is_single_order'] = (frequency_features['total_orders'] == 1).astype(int)

freq_cols = ['Customer ID', 'total_orders', 'total_transactions', 'active_days', 
             'orders_per_month', 'avg_days_between_orders', 'purchase_interval_std', 'is_single_order']
frequency_features = frequency_features[freq_cols]

print("\nFrequency Features Created:")
print("-" * 50)
freq_explanations = {
    'total_orders': 'Total unique orders - Core engagement metric',
    'total_transactions': 'Total line items - Measures basket complexity',
    'active_days': 'Days with at least one purchase',
    'orders_per_month': 'Purchase velocity - Normalized by tenure',
    'avg_days_between_orders': 'Average purchase interval',
    'purchase_interval_std': 'Consistency of purchase timing',
    'is_single_order': 'One-time buyer flag (high risk)'
}
for feat, expl in freq_explanations.items():
    print(f"   {feat}: {expl}")

---

## 3. Monetary Features

**Business Rationale**: Customer value determines retention investment priority. High-value customers justify more retention effort.

In [ ]:
print("="*60)
print("3. MONETARY FEATURES")
print("="*60)

order_values = df_hist.groupby(['Customer ID', 'Invoice']).agg(
    order_revenue=('Revenue', 'sum'),
    order_quantity=('Quantity', 'sum'),
    order_items=('StockCode', 'nunique')
).reset_index()

monetary_features = order_values.groupby('Customer ID').agg(
    total_revenue=('order_revenue', 'sum'),
    
    avg_order_value=('order_revenue', 'mean'),
    
    max_order_value=('order_revenue', 'max'),
    
    min_order_value=('order_revenue', 'min'),
    
    order_value_std=('order_revenue', 'std'),
    
    total_quantity=('order_quantity', 'sum'),
    
    avg_items_per_order=('order_items', 'mean')
).reset_index()

active_days_temp = df_hist.groupby('Customer ID')['InvoiceDate'].apply(lambda x: x.dt.date.nunique()).reset_index()
active_days_temp.columns = ['Customer ID', 'active_days_count']
monetary_features = monetary_features.merge(active_days_temp, on='Customer ID')
monetary_features['revenue_per_active_day'] = monetary_features['total_revenue'] / monetary_features['active_days_count']

monetary_features['order_value_cv'] = monetary_features['order_value_std'] / monetary_features['avg_order_value'].clip(lower=0.01)

monetary_features = monetary_features.drop('active_days_count', axis=1)
monetary_features['order_value_std'] = monetary_features['order_value_std'].fillna(0)
monetary_features['order_value_cv'] = monetary_features['order_value_cv'].fillna(0)

print("\nMonetary Features Created:")
print("-" * 50)
mon_explanations = {
    'total_revenue': 'Total customer spend - Customer lifetime value proxy',
    'avg_order_value': 'Average basket size - Spending tier indicator',
    'max_order_value': 'Largest single order - Peak spending capacity',
    'min_order_value': 'Smallest order - Floor spending behavior',
    'order_value_std': 'Spending variability',
    'total_quantity': 'Total items purchased',
    'avg_items_per_order': 'Basket complexity',
    'revenue_per_active_day': 'Daily spending intensity',
    'order_value_cv': 'Spending consistency (lower = more consistent)'
}
for feat, expl in mon_explanations.items():
    print(f"   {feat}: {expl}")

---

## 4. Trend Features (CRITICAL)

**Business Rationale**: Behavioral changes are early warning signals. Declining engagement predicts churn better than static metrics.

In [ ]:
print("="*60)
print("4. TREND FEATURES (Early Warning Signals)")
print("="*60)

def calculate_trend_features(df_hist, observation_date):
    """Calculate trend features comparing early vs recent behavior"""
    
    customer_dates = df_hist.groupby('Customer ID').agg(
        first_purchase=('InvoiceDate', 'min'),
        last_purchase=('InvoiceDate', 'max')
    ).reset_index()
    
    customer_dates['midpoint'] = customer_dates['first_purchase'] + \
        (customer_dates['last_purchase'] - customer_dates['first_purchase']) / 2
    
    df_trend = df_hist.merge(customer_dates[['Customer ID', 'midpoint', 'first_purchase']], on='Customer ID')
    
    df_trend['period'] = np.where(df_trend['InvoiceDate'] <= df_trend['midpoint'], 'early', 'recent')
    
    period_stats = df_trend.groupby(['Customer ID', 'period']).agg(
        orders=('Invoice', 'nunique'),
        revenue=('Revenue', 'sum'),
        quantity=('Quantity', 'sum')
    ).reset_index()
    
    period_pivot = period_stats.pivot(index='Customer ID', columns='period', values=['orders', 'revenue', 'quantity'])
    period_pivot.columns = [f'{col[0]}_{col[1]}' for col in period_pivot.columns]
    period_pivot = period_pivot.reset_index().fillna(0)
    
    period_pivot['order_trend'] = (period_pivot.get('orders_recent', 0) - period_pivot.get('orders_early', 0)) / \
                                   period_pivot.get('orders_early', 1).clip(lower=1)
    
    period_pivot['revenue_trend'] = (period_pivot.get('revenue_recent', 0) - period_pivot.get('revenue_early', 0)) / \
                                     period_pivot.get('revenue_early', 1).clip(lower=1)
    
    period_pivot['quantity_trend'] = (period_pivot.get('quantity_recent', 0) - period_pivot.get('quantity_early', 0)) / \
                                      period_pivot.get('quantity_early', 1).clip(lower=1)
    
    period_pivot['is_declining'] = ((period_pivot['order_trend'] < 0) | 
                                    (period_pivot['revenue_trend'] < 0)).astype(int)
    
    return period_pivot[['Customer ID', 'order_trend', 'revenue_trend', 'quantity_trend', 'is_declining']]

trend_features = calculate_trend_features(df_hist, OBSERVATION_DATE)

def calculate_recent_trend(df_hist, observation_date):
    """Compare last 30 days vs 31-60 days ago"""
    
    last_30 = df_hist[df_hist['InvoiceDate'] > (observation_date - timedelta(days=30))]
    prev_30 = df_hist[(df_hist['InvoiceDate'] > (observation_date - timedelta(days=60))) & 
                      (df_hist['InvoiceDate'] <= (observation_date - timedelta(days=30)))]
    
    last_30_stats = last_30.groupby('Customer ID').agg(
        orders_last30=('Invoice', 'nunique'),
        revenue_last30=('Revenue', 'sum')
    ).reset_index()
    
    prev_30_stats = prev_30.groupby('Customer ID').agg(
        orders_prev30=('Invoice', 'nunique'),
        revenue_prev30=('Revenue', 'sum')
    ).reset_index()
    
    all_customers = df_hist[['Customer ID']].drop_duplicates()
    recent_trend = all_customers.merge(last_30_stats, on='Customer ID', how='left')
    recent_trend = recent_trend.merge(prev_30_stats, on='Customer ID', how='left')
    recent_trend = recent_trend.fillna(0)
    
    recent_trend['recent_order_momentum'] = recent_trend['orders_last30'] - recent_trend['orders_prev30']
    
    recent_trend['recent_revenue_momentum'] = recent_trend['revenue_last30'] - recent_trend['revenue_prev30']
    
    return recent_trend[['Customer ID', 'orders_last30', 'revenue_last30', 
                         'recent_order_momentum', 'recent_revenue_momentum']]

recent_trend_features = calculate_recent_trend(df_hist, OBSERVATION_DATE)
trend_features = trend_features.merge(recent_trend_features, on='Customer ID', how='left').fillna(0)

print("\nTrend Features Created:")
print("-" * 50)
trend_explanations = {
    'order_trend': 'Change in order frequency (recent vs early) - KEY PREDICTOR',
    'revenue_trend': 'Change in spending (spend decay rate) - KEY PREDICTOR',
    'quantity_trend': 'Change in items purchased',
    'is_declining': 'Binary: any declining behavior signal',
    'orders_last30': 'Orders in most recent 30 days',
    'revenue_last30': 'Revenue in most recent 30 days',
    'recent_order_momentum': 'Order change: last 30d vs prev 30d',
    'recent_revenue_momentum': 'Revenue change: last 30d vs prev 30d'
}
for feat, expl in trend_explanations.items():
    print(f"   {feat}: {expl}")

---

## 5. Engagement Features

**Business Rationale**: Product diversity and basket patterns indicate engagement depth and switching costs.

In [ ]:
print("="*60)
print("5. ENGAGEMENT FEATURES")
print("="*60)

engagement_features = df_hist.groupby('Customer ID').agg(
    unique_products=('StockCode', 'nunique'),
    
    unique_descriptions=('Description', 'nunique'),
    
    total_line_items=('Invoice', 'count')
).reset_index()

products_per_order = df_hist.groupby(['Customer ID', 'Invoice'])['StockCode'].nunique().reset_index()
products_per_order = products_per_order.groupby('Customer ID')['StockCode'].mean().reset_index()
products_per_order.columns = ['Customer ID', 'avg_products_per_order']
engagement_features = engagement_features.merge(products_per_order, on='Customer ID')

product_counts = df_hist.groupby(['Customer ID', 'StockCode']).size().reset_index(name='product_purchases')
product_concentration = product_counts.groupby('Customer ID')['product_purchases'].agg(['max', 'sum']).reset_index()
product_concentration.columns = ['Customer ID', 'max_product_purchases', 'total_product_purchases']
product_concentration['product_concentration'] = product_concentration['max_product_purchases'] / product_concentration['total_product_purchases']
engagement_features = engagement_features.merge(product_concentration[['Customer ID', 'product_concentration']], on='Customer ID')

customer_country = df_hist.groupby('Customer ID')['Country'].first().reset_index()
customer_country['is_uk'] = (customer_country['Country'] == 'United Kingdom').astype(int)
engagement_features = engagement_features.merge(customer_country[['Customer ID', 'is_uk']], on='Customer ID')

print("\nEngagement Features Created:")
print("-" * 50)
eng_explanations = {
    'unique_products': 'Product diversity - Higher = more engaged',
    'unique_descriptions': 'Category diversity',
    'total_line_items': 'Total transaction count',
    'avg_products_per_order': 'Basket complexity - Exploration behavior',
    'product_concentration': 'Repeat purchase focus (0-1, higher = more focused)',
    'is_uk': 'Domestic vs international customer'
}
for feat, expl in eng_explanations.items():
    print(f"   {feat}: {expl}")

---

## 6. Combine All Features

In [ ]:
print("="*60)
print("COMBINING ALL FEATURES")
print("="*60)

features_df = recency_features[['Customer ID', 'recency_days', 'tenure_days', 
                                 'active_period_days', 'recency_ratio', 'is_recent_30d']].copy()

features_df = features_df.merge(frequency_features, on='Customer ID', how='left')

features_df = features_df.merge(monetary_features, on='Customer ID', how='left')

features_df = features_df.merge(trend_features, on='Customer ID', how='left')

features_df = features_df.merge(engagement_features, on='Customer ID', how='left')

print(f"\n✅ Combined feature dataset:")
print(f"   Customers: {len(features_df):,}")
print(f"   Features: {len(features_df.columns) - 1}")

In [ ]:
print("\n📊 MISSING VALUES ANALYSIS")
print("="*60)

missing = features_df.isnull().sum()
missing_pct = (missing / len(features_df) * 100).round(2)
missing_df = pd.DataFrame({'Missing': missing, 'Pct': missing_pct})
missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
    
    print("\n→ Filling missing values...")
    
    features_df['purchase_interval_std'] = features_df['purchase_interval_std'].fillna(0)
    features_df['order_value_std'] = features_df['order_value_std'].fillna(0)
    features_df['order_value_cv'] = features_df['order_value_cv'].fillna(0)
    
    numeric_cols = features_df.select_dtypes(include=[np.number]).columns
    features_df[numeric_cols] = features_df[numeric_cols].fillna(0)
else:
    print("No missing values!")

print(f"\n✅ Final missing values: {features_df.isnull().sum().sum()}")

In [ ]:
print("\n📊 FEATURE SUMMARY STATISTICS")
print("="*60)

feature_cols = [c for c in features_df.columns if c != 'Customer ID']
features_df[feature_cols].describe().T

---

## 7. Feature Rationale Document

In [ ]:
print("="*80)
print("                    FEATURE RATIONALE DOCUMENT")
print("="*80)

feature_documentation = """
┌──────────────────────────────────────────────────────────────────────────────┐
│                         RECENCY FEATURES                                     │
├──────────────────────────────────────────────────────────────────────────────┤
│ recency_days          │ Days since last purchase. THE strongest churn       │
│                       │ predictor. Higher = more likely to churn.           │
├───────────────────────┼──────────────────────────────────────────────────────┤
│ tenure_days           │ Days since first purchase. Longer tenure indicates  │
│                       │ more invested customers who are less likely to      │
│                       │ churn.                                              │
├───────────────────────┼──────────────────────────────────────────────────────┤
│ recency_ratio         │ Recency normalized by tenure. A customer with 100   │
│                       │ day tenure and 50 day recency has same ratio as     │
│                       │ 200 day tenure and 100 day recency.                 │
└──────────────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                        FREQUENCY FEATURES                                    │
├──────────────────────────────────────────────────────────────────────────────┤
│ total_orders          │ Total unique orders. More orders = more engaged     │
│                       │ = less likely to churn. Critical for identifying    │
│                       │ one-time buyers (highest churn risk).               │
├───────────────────────┼──────────────────────────────────────────────────────┤
│ orders_per_month      │ Purchase velocity normalized by customer age.       │
│                       │ Accounts for customers at different lifecycle       │
│                       │ stages.                                             │
├───────────────────────┼──────────────────────────────────────────────────────┤
│ purchase_interval_std │ Consistency of purchase timing. Lower std = more    │
│                       │ regular buyer = less likely to churn unexpectedly.  │
└──────────────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                        MONETARY FEATURES                                     │
├──────────────────────────────────────────────────────────────────────────────┤
│ total_revenue         │ Customer lifetime value proxy. Higher value =       │
│                       │ more invested in relationship = less likely to      │
│                       │ churn. Also determines retention investment.        │
├───────────────────────┼──────────────────────────────────────────────────────┤
│ avg_order_value       │ Spending tier indicator. Premium customers behave   │
│                       │ differently than budget customers.                  │
├───────────────────────┼──────────────────────────────────────────────────────┤
│ order_value_cv        │ Coefficient of variation of order values.           │
│                       │ Consistent spenders (low CV) are more predictable.  │
└──────────────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                    TREND FEATURES (CRITICAL!)                                │
├──────────────────────────────────────────────────────────────────────────────┤
│ order_trend           │ Change in purchase frequency between first half     │
│                       │ and second half of customer journey. Negative =     │
│                       │ declining engagement = EARLY WARNING SIGNAL.        │
├───────────────────────┼──────────────────────────────────────────────────────┤
│ revenue_trend         │ Change in spending (spend decay rate). Negative     │
│                       │ trend indicates customer is disengaging before      │
│                       │ full churn.                                         │
├───────────────────────┼──────────────────────────────────────────────────────┤
│ recent_order_momentum │ Orders in last 30 days vs previous 30 days.         │
│                       │ Captures very recent behavioral shifts.             │
└──────────────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                       ENGAGEMENT FEATURES                                    │
├──────────────────────────────────────────────────────────────────────────────┤
│ unique_products       │ Product diversity. More products explored =         │
│                       │ deeper engagement = higher switching costs.         │
├───────────────────────┼──────────────────────────────────────────────────────┤
│ product_concentration │ Do they buy same products repeatedly? High          │
│                       │ concentration = habitual buyer (good for            │
│                       │ retention but vulnerable if product unavailable).   │
└──────────────────────────────────────────────────────────────────────────────┘
"""
print(feature_documentation)

---

## 8. Save Features

In [ ]:
output_path = os.path.join(PROCESSED_DATA_PATH, 'customer_features.parquet')
features_df.to_parquet(output_path, index=False)
print(f"\n✅ Features saved to: {output_path}")
print(f"   Shape: {features_df.shape}")

In [ ]:
print("\n" + "="*60)
print("              FEATURE ENGINEERING SUMMARY")
print("="*60)

feature_categories = {
    'Recency': ['recency_days', 'tenure_days', 'active_period_days', 'recency_ratio', 'is_recent_30d'],
    'Frequency': ['total_orders', 'total_transactions', 'active_days', 'orders_per_month', 
                  'avg_days_between_orders', 'purchase_interval_std', 'is_single_order'],
    'Monetary': ['total_revenue', 'avg_order_value', 'max_order_value', 'min_order_value',
                 'order_value_std', 'total_quantity', 'avg_items_per_order', 
                 'revenue_per_active_day', 'order_value_cv'],
    'Trend': ['order_trend', 'revenue_trend', 'quantity_trend', 'is_declining',
              'orders_last30', 'revenue_last30', 'recent_order_momentum', 'recent_revenue_momentum'],
    'Engagement': ['unique_products', 'unique_descriptions', 'total_line_items',
                   'avg_products_per_order', 'product_concentration', 'is_uk']
}

total_features = 0
for category, features in feature_categories.items():
    print(f"\n{category} Features: {len(features)}")
    for f in features:
        if f in features_df.columns:
            print(f"   ✓ {f}")
            total_features += 1

print(f"\n{'='*60}")
print(f"TOTAL FEATURES: {total_features}")
print(f"{'='*60}")

---

##  Phase 3 Checklist

- [x] Created Recency features (5 features)
- [x] Created Frequency features (7 features)
- [x] Created Monetary features (9 features)
- [x] Created Trend features (8 features) - KEY PREDICTORS
- [x] Created Engagement features (6 features)
- [x] Documented business rationale for each feature
- [x] Handled missing values
- [x] Saved feature dataset

**Phase 3 Status: COMPLETE** 

---

**Next**: Phase 4 - Label Creation & Data Leakage Control